In [ ]:
import gymnasium as gym 
import torch 
import torch.nn as nn
env  = gym.make("CartPole-v1")
obs , info = env.reset(seed=42)
obs = torch.tensor(obs) 

class Policy(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.core = nn.Sequential(
            nn.Linear(4,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,2),
           
            
        )
    def forward(self, x):
        return self.core(x)
    
policy = Policy()

episodes = 40000
gamma = 0.99
optimizer = torch.optim.Adam(policy.parameters(), lr=0.0001)
beta = 0.001

def is_terminal(observation):
    x,v,theta , omega = observation 
    x_terminal = x <= -4.8 or x>=4.8
    theta_terminal = theta <= -0.209*7.5 or theta >= 0.209*7.5
    return x_terminal or theta_terminal




total_loss = 0 
for episode in range(episodes):
    obs , info = env.reset()
    obs = torch.tensor(obs,dtype=torch.float32)
    done = False 
    observations = []
    actions = []
    rewards = []
    
    while not done :
        logits = policy(obs)
        probs = torch.softmax(logits , dim=-1)
        action = torch.multinomial(probs,num_samples=1).item()
        observations.append(obs)
        actions.append(action)
        
        obs , reward , terminated , truncated , info = env.step (action)
        obs = torch.tensor(obs,dtype=torch.float32)
        rewards.append(reward)
        
        done = is_terminal(obs) or truncated
    Gs = []
    G = 0
    for i in range(len(rewards) - 1 , -1 , -1):
        G = rewards[i] + gamma * G
        Gs.append(G)
    Gs.reverse()
    Gs = torch.tensor(Gs,dtype=torch.float32)
    Gs = (Gs - Gs.mean()) / (Gs.std(unbiased=False) + 1e-8)

    loss = 0 
    for obs, action , G in zip(observations , actions , Gs):
        logits = policy(obs)
        probs = torch.softmax(logits, dim =-1)
        entropy = -torch.sum(probs * torch.log(probs + 1e-8))
        max_prob = probs[action]
      
        log_prob = torch.log(max_prob)
        
        loss += -log_prob * G  - beta * entropy
    
    loss /= len(observations)
    total_loss += loss
    if (episode + 1) % 100 ==0:
        
         
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(policy.parameters(), max_norm=1.0)
        optimizer.step()
        print(f"Episode {episode} , Loss: {total_loss/100}")
        total_loss = 0 
        optimizer.zero_grad()       

c:\Users\beca\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\classic_control\cartpole.py:214: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


Episode 99 , Loss: -0.0009240620420314372
Episode 199 , Loss: -0.00029889700817875564
Episode 299 , Loss: -0.0007885179948061705
Episode 399 , Loss: -0.001028257654979825
Episode 499 , Loss: 0.0007567774737253785
Episode 599 , Loss: -0.000719390285667032
Episode 699 , Loss: -0.0001845820079324767
Episode 799 , Loss: -0.0004077876219525933
Episode 899 , Loss: 0.00027809274615719914
Episode 999 , Loss: -0.0018409399781376123
Episode 1099 , Loss: -0.0009994808351621032
Episode 1199 , Loss: 8.264217467512935e-05
Episode 1299 , Loss: -0.0008205248159356415
Episode 1399 , Loss: -0.0008279357571154833
Episode 1499 , Loss: -0.001042317831888795
Episode 1599 , Loss: -5.6851138651836663e-05
Episode 1699 , Loss: -0.0016341751907020807
Episode 1799 , Loss: -0.0011177374981343746
Episode 1899 , Loss: -0.0010056121973320842
Episode 1999 , Loss: -0.0009555606520734727
Episode 2099 , Loss: -0.0011415805201977491
Episode 2199 , Loss: -0.00036974361864849925
Episode 2299 , Loss: -0.001179000362753868
Ep

KeyboardInterrupt: 

In [ ]:
env  = gym.make("CartPole-v1")
obs , info = env.reset(seed=42)
obs = torch.tensor(obs) 
total_loss = 0 
for episode in range(episodes):
    obs , info = env.reset()
    obs = torch.tensor(obs,dtype=torch.float32)
    done = False 
    observations = []
    actions = []
    rewards = []
    
    while not done :
        logits = policy(obs)
        probs = torch.softmax(logits , dim=-1)
        action = torch.multinomial(probs,num_samples=1).item()
        observations.append(obs)
        actions.append(action)
        
        obs , reward , terminated , truncated , info = env.step (action)
        obs = torch.tensor(obs,dtype=torch.float32)
        rewards.append(reward)
        
        done = is_terminal(obs) or truncated
    Gs = []
    G = 0
    for i in range(len(rewards) - 1 , -1 , -1):
        G = rewards[i] + gamma * G
        Gs.append(G)
    Gs.reverse()
    Gs = torch.tensor(Gs,dtype=torch.float32)
    Gs = (Gs - Gs.mean()) / (Gs.std(unbiased=False) + 1e-8)

    loss = 0 
    for obs, action , G in zip(observations , actions , Gs):
        logits = policy(obs)
        probs = torch.softmax(logits, dim =-1)
        entropy = -torch.sum(probs * torch.log(probs + 1e-8))
        max_prob = probs[action]
      
        log_prob = torch.log(max_prob)
        
        loss += -log_prob * G  - beta * entropy
    
    loss /= len(observations)
    total_loss += loss
    if (episode + 1) % 100 ==0:
        
         
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(policy.parameters(), max_norm=1.0)
        optimizer.step()
        print(f"Episode {episode} , Loss: {total_loss/100}")
        total_loss = 0 
        optimizer.zero_grad()   
       

        
    
   



        

Episode 99 , Loss: 0.03374747186899185


In [14]:
import gymnasium as gym 
import torch 
import torch.nn as nn
env  = gym.make("CartPole-v1",render_mode="human")
obs , info = env.reset(seed=42)
obs = torch.tensor(obs) 




with torch.no_grad():
        
    for episode in range(100000):
        obs , info = env.reset()
        obs = torch.tensor(obs,dtype=torch.float32)
        done = False 
        observations = []
        actions = []
        rewards = []
        
        while not done :
            logits = policy(obs)
            probs = torch.softmax(logits , dim=-1)
            action = torch.multinomial(probs,num_samples=1).item()
            observations.append(obs)
            actions.append(action)
            
            obs , reward , terminated , truncated , info = env.step (action)
            obs = torch.tensor(obs,dtype=torch.float32)
            rewards.append(reward)
            
            done = is_terminal(obs) or truncated
    # Gs = []
    # G = 0
    # for i in range(len(rewards) - 1 , -1 , -1):
    #     G = rewards[i] + gamma * G
    #     Gs.append(G)
    # Gs.reverse()
    # Gs = torch.tensor(Gs,dtype=torch.float32)
    # Gs = (Gs - Gs.mean()) / (Gs.std(unbiased=False) + 1e-8)

    # loss = 0 
    # for obs, action , G in zip(observations , actions , Gs):
    #     logits = policy(obs)
    #     probs = torch.softmax(logits, dim =-1)
    #     entropy = -torch.sum(probs * torch.log(probs + 1e-8))
    #     max_prob = probs[action]
      
    #     log_prob = torch.log(max_prob)
        
    #     loss += -log_prob * G  - beta * entropy
    # # print(f"Episode {episode} , Loss: {loss.item()}")
    # loss /= len(observations)
    # optimizer.zero_grad()    
    # loss.backward()
    
    # torch.nn.utils.clip_grad_norm_(policy.parameters(), max_norm=1.0)
    # optimizer.step()
   
        





        

KeyboardInterrupt: 